In [1]:
import sklearn
import pandas as pd
pd.set_option('display.float_format',lambda x: '%.4f' %x)
import seaborn as sns
sns.set_context('paper', font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import math
%matplotlib inline

import math
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras import Dense
from tensorflow.keras import LSTM
from tensorflow.keras import Dropout
from tensorflow.keras import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow.


dataframe=pd.read_csv('dengue_features_train.csv')
labels=pd.read_csv('dengue_labels_train.csv')
target_df=pd.read_csv('dengue_features_test.csv')
submission_format=pd.read_csv('submission_format.csv')
target_df['month']=[int(i[5:7]) for i in target_df['week_start_date']]

def clean_data(df):
    #file.set_index('week_start_date', drop = True, inplace = True)
    df['month']=[int(i[5:7]) for i in df['week_start_date']]
    #df['nvdi']= df['ndvi_ne']+df['ndvi_nw']+df['ndvi_se']+df['ndvi_sw']
    
    df=df.drop(['city','week_start_date', 'year', 'weekofyear'], axis = 1, inplace = True)
    
    #,'ndvi_ne','ndvi_nw', 'ndvi_se', 'ndvi_sw'

def fill_missing_values(df):
    imputer = SimpleImputer(strategy="median") 
    imputer.fit(df)
    x=imputer.transform(df)
   
    return pd.DataFrame(x, columns=df.columns)

def split_data_in_cities(df):
    #cities=set(df['city'])
    cities=['sj','iq']
    new_files=[]
    for city in list(cities):
        new_files.append(df[df['city'] == city])

    return new_files

def eliminate_not_outliers(df_labels, city_df):
    cases=np.array(df_labels['total_cases'])
    df_labels=df_labels.reset_index()
    city_df=city_df.reset_index()
    rows_to_eliminate=[]
    mean=cases.mean()
    for i in range(len(cases)):
        if(cases[i]<(mean+20)):
            rows_to_eliminate.append(i)
    
    #print(rows_to_eliminate)
    #print(df_labels)
    df_labels.drop(rows_to_eliminate, axis=0, inplace=True)
    city_df.drop(rows_to_eliminate, axis=0, inplace=True)
    df_labels.drop(['index'], axis=1, inplace=True)
    city_df.drop(['index'], axis=1, inplace=True)
    return df_labels, city_df


new_files=split_data_in_cities(dataframe)
new_labels=split_data_in_cities(labels)
new_target=split_data_in_cities(target_df)

sj_df=new_files[0]
iq_df=new_files[1]

sj_labels=new_labels[0]
iq_labels=new_labels[1]

sj_target=new_target[0]
iq_target=new_target[1]

clean_data(sj_df)
clean_data(iq_df)
sj_labels, sj_df =eliminate_not_outliers(sj_labels, sj_df)
iq_labels, iq_df= eliminate_not_outliers(iq_labels, iq_df)

sj_df=fill_missing_values(sj_df)
iq_df=fill_missing_values(iq_df)

clean_data(dataframe)
dataframe=fill_missing_values(dataframe)


/home/joramas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/joramas/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,month
0,0.199271,0.159243,0.258129,0.227714,21.96,298.297143,299.714286,295.585714,308.8,292.1,...,86.998571,21.96,17.185714,10.657143,28.000000,11.633333,34.2,22.0,4.5,10.0
1,0.329086,0.286214,0.294057,0.315929,89.77,298.518571,300.242857,296.900000,307.3,294.9,...,91.944286,89.77,18.602857,9.028571,27.820000,10.700000,34.1,22.5,34.1,10.0
2,0.300800,0.252800,0.344957,0.348329,49.69,298.324286,299.657143,296.485714,306.8,294.3,...,90.662857,49.69,18.105714,8.000000,28.500000,9.750000,33.7,22.9,46.0,12.0
3,0.268029,0.142429,0.288671,0.185071,35.02,297.884286,298.950000,296.390000,307.4,294.2,...,92.614286,35.02,18.004286,7.928571,27.650000,9.450000,34.0,22.9,57.2,12.0
4,0.346686,0.315414,0.270857,0.374200,74.68,297.168571,297.707143,296.484286,304.2,292.4,...,96.354286,74.68,18.121429,5.928571,27.200000,8.900000,33.7,22.4,184.2,11.0
5,0.227871,0.194500,0.190914,0.209557,74.08,299.418571,301.235714,296.097143,309.3,293.6,...,84.611429,74.08,17.738571,11.642857,28.250000,11.000000,35.8,22.5,107.4,11.0
6,0.201943,0.159367,0.159043,0.241086,53.01,299.177143,300.378571,296.440000,309.5,293.3,...,87.130000,53.01,18.110000,10.957143,27.900000,11.000000,34.1,22.4,64.0,12.0
7,0.323429,0.309514,0.275829,0.335900,70.04,297.787143,298.835714,296.574286,305.8,294.5,...,93.938571,70.04,18.207143,6.500000,27.820000,10.700000,34.1,21.9,9.9,12.0
8,0.261143,0.203314,0.243743,0.236357,49.56,298.701429,300.035714,296.385714,308.5,293.8,...,88.780000,49.56,18.040000,10.214286,28.375000,9.650000,34.0,22.6,128.3,12.0
9,0.151767,0.137700,0.147800,0.160783,49.32,297.188571,298.292857,295.777143,304.8,294.4,...,93.114286,49.32,17.364286,6.214286,27.300000,8.950000,33.5,22.3,250.9,1.0


In [2]:
x_sj_train_spikes, x_sj_test_spikes, y_sj_train_spikes, y_sj_test_spikes = train_test_split(sj_df, np.array(sj_labels['is_spike']), test_size=0.2)
x_iq_train_spikes, x_iq_test_spikes, y_iq_train_spikes, y_iq_test_spikes = train_test_split(iq_df, np.array(iq_labels['is_spike']), test_size=0.2)
x_train_spikes, x_test_spikes, y_train_spikes, y_test_spikes = train_test_split(dataframe, np.array(labels['is_spike']), test_size=0.2)